In this course project, I will build a regression model using the deep learning Keras library, and then I will experiment with increasing the number of training epochs and changing the number of hidden layers, and we will see how changing these parameters impacts the performance of the model.

<strong>1. Assignment Topic:</strong>

In this project, I will build a regression model using the Keras library to model the same data about concrete compressive strength that we used in Module 3.

<strong>2. Concrete Data:</strong>

The data can be found here again: https://cocl.us/concrete_data.

To recap, the predictors in the data of concrete strength include:

* Cement
* Blast Furnace Slag
* Fly Ash
* Water
* Superplasticizer
* Coarse Aggregate
* Fine Aggregate

# Regression Models with Keras

Start by importing the pandas, Keras, Numpy libraries.

In [1]:
import pandas as pd
import numpy as np
import keras 

import warnings
warnings.simplefilter('ignore', FutureWarning)

2025-02-05 07:06:20.371584: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-05 07:06:20.496872: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Download the data and read it into a <em>pandas</em> dataframe.

In [2]:
filepath = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.


In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

In [7]:
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Quick sanity check of the predictors and the target dataframes.

In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Let's save the number of predictors to *n_cols* since we will need this number when building our network.

In [10]:
n_cols = predictors.shape[1] # number of predictors
n_cols

8

##  Import Keras Packages

##### Import the rest of the packages from the Keras library that will be needed to build regression model.

In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

## Build a Neural Network

I will create a function that defines our regression model's requirements for Part A so that I can conveniently call it to create our model.


### A. Build a baseline model

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error  as the loss function.

In [12]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function creates a model that has one hidden layer with 10 neurons and a ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function.

### Import scikit-learn in order to randomly split the data into a training and test sets

1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_split helper function from Scikit-learn.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [21]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42) #random_state set based on example from scikit-learn doc

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Train and Test the Network

Call the function now to create our model.

In [22]:
# build the model
model = regression_model()

/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Now train the model based on the requirements.

2. Train the model on the training data using 50 epochs.

In [23]:
# fit the model
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=2)

Epoch 1/50
23/23 - 1s - 34ms/step - loss: 1683.7609
Epoch 2/50
23/23 - 0s - 5ms/step - loss: 1667.9861
Epoch 3/50
23/23 - 0s - 6ms/step - loss: 1654.0475
Epoch 4/50
23/23 - 0s - 6ms/step - loss: 1641.8647
Epoch 5/50
23/23 - 0s - 5ms/step - loss: 1631.1055
Epoch 6/50
23/23 - 0s - 4ms/step - loss: 1621.6832
Epoch 7/50
23/23 - 0s - 5ms/step - loss: 1613.2493
Epoch 8/50
23/23 - 0s - 4ms/step - loss: 1605.7983
Epoch 9/50
23/23 - 0s - 4ms/step - loss: 1599.0630
Epoch 10/50
23/23 - 0s - 4ms/step - loss: 1592.8916
Epoch 11/50
23/23 - 0s - 4ms/step - loss: 1587.2719
Epoch 12/50
23/23 - 0s - 4ms/step - loss: 1581.9194
Epoch 13/50
23/23 - 0s - 4ms/step - loss: 1576.7540
Epoch 14/50
23/23 - 0s - 4ms/step - loss: 1571.4023
Epoch 15/50
23/23 - 0s - 4ms/step - loss: 1565.6338
Epoch 16/50
23/23 - 0s - 4ms/step - loss: 1559.2990
Epoch 17/50
23/23 - 0s - 4ms/step - loss: 1551.8206
Epoch 18/50
23/23 - 0s - 5ms/step - loss: 1543.4133
Epoch 19/50
23/23 - 0s - 4ms/step - loss: 1533.6385
Epoch 20/50
23/23 - 

Next evaluate the model on the test data according the requirements #3.


3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

In [24]:
model_eval = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
model_eval

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 741.2422 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


742.015380859375

Now compute the mean squared error between the predicted concrete strength and the actual concrete strength.

Import the mean_squared_error function from Scikit-learn.

In [25]:
from sklearn.metrics import mean_squared_error

In [26]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean_mse = np.mean(mean_square_error)
standard_deviation_mse = np.std(mean_square_error)

print(f'Mean Square Error: {mean_square_error}')
print(f'Mean : {mean_mse}\nStandard Deviation: {standard_deviation_mse}')    


Mean Square Error: 742.0153690405042
Mean : 742.0153690405042
Standard Deviation: 0.0


Finally finish the last requirements

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.
5. Report the mean and the standard deviation of the mean squared errors.

In [27]:
mean_squared_errors = []

scaler = StandardScaler()

for i in range(1, 51):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    model_eval = model.evaluate(X_test, y_test)
    print(f'Model evaluation {i}: {model_eval}')
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean_mse = np.mean(mean_squared_errors)
standard_deviation_mse = np.std(mean_squared_errors)

print(f'Mean : {mean_mse}\nStandard Deviation: {standard_deviation_mse}')

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 200.7211 
Model evaluation 1: 204.2762908935547
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 114.6496
Model evaluation 2: 118.93869018554688
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 105.7245 
Model evaluation 3: 99.23819732666016
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 77.5638 
Model evaluation 4: 71.4303207397461
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 72.1203 
Model evaluation 5: 69.8970718383789
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 56.8671 
Model evaluation 6: 62.212379455566406
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 40.7239 
Model evaluation 7: 40.04090118408203
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 44.6400 
Mode